In [158]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [159]:
spark = SparkSession.builder.appName("spark ML").getOrCreate()
sc = spark.sparkContext

# Load data

In [160]:
training_data = spark.read.csv("./training_data.csv", sep=';', header=True)

# EDA and features engineering

In [161]:
training_data.show()

+---+----+-----+-----+---+---+------+---+----+-----------------+------------+--------------+---------+----------+-------+
| V1|  V2|   V3|   V4| V5| V6|    V7| V8| PDC|          Contrat|    localite|MeterMatricule|Millesime|        T0|  Label|
+---+----+-----+-----+---+---+------+---+----+-----------------+------------+--------------+---------+----------+-------+
|152|0.99|0.897| 1.67|  5| 54| 37743|  3|  11|     LIANCOURTOIS|LIANCOURTOIS|            NA|    C17FA|08/12/2017|STOPPED|
| 76|   1|0.993|    0|  0|239|413425|  1|1701|NEUVY-LES-MOULINS|       NEUVY|            NA|    C13FA|29/10/2016|STOPPED|
| 71|0.32|0.965| 8.38| 53| 45| 85193|  8|1802|NEUVY-LES-MOULINS|       NEUVY|            NA|    C13FA|28/11/2016|STOPPED|
| 92|0.84|0.871| 4.61| 77| 49|263697| 36|2201|NEUVY-LES-MOULINS|       NEUVY|            NA|    C13FA|29/10/2017|STOPPED|
| 91|0.31|0.189|28.95|126| 13| 90206| 27|5201|NEUVY-LES-MOULINS|       NEUVY|            NA|    C12FA|19/08/2018|STOPPED|
| 65|0.74|0.451|12.57| 6

In [162]:
training_data.count()

169078

In [163]:
training_data.groupBy("Label").agg(f.count("*")).show()

+-------+--------+
|  Label|count(1)|
+-------+--------+
|BLOCKED|    1364|
|STOPPED|  167714|
+-------+--------+



In [164]:
training_data.printSchema()

root
 |-- V1: string (nullable = true)
 |-- V2: string (nullable = true)
 |-- V3: string (nullable = true)
 |-- V4: string (nullable = true)
 |-- V5: string (nullable = true)
 |-- V6: string (nullable = true)
 |-- V7: string (nullable = true)
 |-- V8: string (nullable = true)
 |-- PDC: string (nullable = true)
 |-- Contrat: string (nullable = true)
 |-- localite: string (nullable = true)
 |-- MeterMatricule: string (nullable = true)
 |-- Millesime: string (nullable = true)
 |-- T0: string (nullable = true)
 |-- Label: string (nullable = true)



## compute V9 from V7 and Millesime features

In [165]:
consumption_features = ["V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9"]
millesime_0 = ['C10FA','C10LA','C10SA', 'C11FA','C11LA','C11SA']
millesime_1 = ['D16BU', 'Z12ER', 'C07AA']

In [166]:
training_data = training_data.withColumn(
    'V9',
    f.when(
        (training_data['Millesime'].isin(millesime_0)) & (f.length(training_data['V7'])>3),
        0
    ).otherwise(
        f.when(
            training_data['Millesime'].isin(millesime_1),
            1
        ).otherwise(2)
    )
)

In [167]:
training_data.select(['V7','Millesime', 'V9']).where(training_data.V9 == 0).show(5)

+------+---------+---+
|    V7|Millesime| V9|
+------+---------+---+
|135665|    C11FA|  0|
|410287|    C11FA|  0|
|187007|    C11FA|  0|
|350368|    C11FA|  0|
|314965|    C11FA|  0|
+------+---------+---+
only showing top 5 rows



In [168]:
training_data.select("V9").distinct().collect()

[Row(V9=1), Row(V9=2), Row(V9=0)]

# Target values count by locality type

In [169]:
agg_label_locality = training_data.groupBy(["localite", "Label"]).agg(f.count("*"))
agg_label_locality.show()

+------------+-------+--------+
|    localite|  Label|count(1)|
+------------+-------+--------+
| LEMANSMETRO|STOPPED|    1049|
|LIANCOURTOIS|STOPPED|     562|
|   LYONNAISE|BLOCKED|    1153|
|     OCEAVHF|BLOCKED|     190|
|       SIEVA|STOPPED|    1245|
|  CHAMPFLEUR|STOPPED|     178|
|    OMFLEURY|STOPPED|     231|
|       NEUVY|STOPPED|      34|
|       CREMA|STOPPED|       2|
|      SIDERM|BLOCKED|       8|
|  CHAMPFLEUR|BLOCKED|       1|
|    FLORENCE|STOPPED|      54|
|   CALEDONIE|STOPPED|       1|
|    AGUR-LCF|STOPPED|    2201|
|     OCEAVHF|STOPPED|   39792|
|      AREZZO|STOPPED|       1|
|   POLYNESIE|STOPPED|       2|
|     NICEREA|STOPPED|       3|
|    LAFLECHE|STOPPED|     290|
|MONTDEMARSAN|STOPPED|     652|
+------------+-------+--------+
only showing top 20 rows



In [170]:
training_data.filter(
    (training_data.localite == "LYONNAISE") 
    &
    (training_data.Label == "BLOCKED")
).count()

1153

### defining a function for column casting

In [171]:
def cast_col(df, column, type="string"):
    if isinstance(column, list):
        for c in column:
            df = df.withColumn(c, df[c].cast(type))
        return df
    else:
        return df.withColumn(column, df[column].cast(type))

In [172]:
training_data = cast_col(training_data, consumption_features, "integer")
training_data.printSchema()

root
 |-- V1: integer (nullable = true)
 |-- V2: integer (nullable = true)
 |-- V3: integer (nullable = true)
 |-- V4: integer (nullable = true)
 |-- V5: integer (nullable = true)
 |-- V6: integer (nullable = true)
 |-- V7: integer (nullable = true)
 |-- V8: integer (nullable = true)
 |-- PDC: string (nullable = true)
 |-- Contrat: string (nullable = true)
 |-- localite: string (nullable = true)
 |-- MeterMatricule: string (nullable = true)
 |-- Millesime: string (nullable = true)
 |-- T0: string (nullable = true)
 |-- Label: string (nullable = true)
 |-- V9: integer (nullable = false)



In [173]:
consumption_features_df = training_data.select(consumption_features+["Label"])
consumption_features_df.show()

+---+---+---+---+---+---+------+---+---+-------+
| V1| V2| V3| V4| V5| V6|    V7| V8| V9|  Label|
+---+---+---+---+---+---+------+---+---+-------+
|152|  0|  0|  1|  5| 54| 37743|  3|  2|STOPPED|
| 76|  1|  0|  0|  0|239|413425|  1|  2|STOPPED|
| 71|  0|  0|  8| 53| 45| 85193|  8|  2|STOPPED|
| 92|  0|  0|  4| 77| 49|263697| 36|  2|STOPPED|
| 91|  0|  0| 28|126| 13| 90206| 27|  2|STOPPED|
| 65|  0|  0| 12| 60| 33|187459|  7|  2|STOPPED|
| 63|  0|  0| 43|166| 21|185199|  8|  2|STOPPED|
| 62|  0|  0|215|426|  2|  5338|  2|  2|STOPPED|
|122|  0|  0| 38|114|191|113709|  3|  2|STOPPED|
|238|  0|  0|  5| 19|149|103712|  5|  2|STOPPED|
| 69|  0|  0| 13| 37|220| 89662|  9|  2|STOPPED|
| 77|  0|  0|  9| 82| 83| 51327| 15|  2|STOPPED|
| 85|  0|  0| 86|188| 15| 19088|  3|  2|STOPPED|
| 80|  0|  0|  6| 73| 40| 36604| 22|  2|STOPPED|
|120|  0|  0|  3| 20| 78| 24936|  7|  2|STOPPED|
|146|  0|  0| 83|160| 13| 10296|  3|  2|STOPPED|
|111|  0|  0|  7| 54| 64|162579| 27|  2|STOPPED|
| 69|  0|  0|  9| 65

# Modelisation:


In [174]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [175]:
training_data.groupBy("Label").agg(f.count("*")).show()

+-------+--------+
|  Label|count(1)|
+-------+--------+
|BLOCKED|    1364|
|STOPPED|  167714|
+-------+--------+



### We deal with imbalanced dataset.
let's balanced it before going ahead for model training

In [176]:
blocked_data = training_data.filter(f.col("Label") == "BLOCKED")
stopped_data = training_data.filter(f.col("Label") == "STOPPED")

# Subsample the STOPPED class
stopped_data_sampled = stopped_data.sample(withReplacement=False, fraction=0.02, seed=2000)

balanced_data = blocked_data.union(stopped_data_sampled)

In [177]:
assembler = VectorAssembler(inputCols=consumption_features, outputCol="features")
balanced_data = assembler.transform(balanced_data)

label_indexer = StringIndexer(inputCol="Label", outputCol="label")
balanced_data = label_indexer.fit(balanced_data).transform(balanced_data)

## label encoding:
- BLOCKED: 0  
- STOPPED: 1

In [178]:
data = balanced_data.select("features", "label")
data.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[109.0,0.0,0.0,2....|  1.0|
|[138.0,0.0,0.0,0....|  1.0|
|(9,[0,6],[258.0,1...|  1.0|
|[73.0,0.0,0.0,0.0...|  1.0|
|[274.0,1.0,0.0,0....|  1.0|
|[72.0,0.0,0.0,102...|  1.0|
|(9,[0,6,8],[181.0...|  1.0|
|[178.0,1.0,1.0,0....|  1.0|
|[25.0,0.0,0.0,4.0...|  1.0|
|[95.0,1.0,0.0,2.0...|  1.0|
|[100.0,0.0,0.0,10...|  1.0|
|[52.0,0.0,0.0,0.0...|  1.0|
|(9,[0,6],[279.0,4...|  1.0|
|(9,[0,6],[279.0,5...|  1.0|
|[119.0,0.0,0.0,7....|  1.0|
|(9,[0,6],[279.0,1...|  1.0|
|[116.0,0.0,0.0,2....|  1.0|
|[71.0,1.0,0.0,2.0...|  1.0|
|[162.0,0.0,0.0,8....|  1.0|
|[257.0,0.0,0.0,8....|  1.0|
+--------------------+-----+
only showing top 20 rows



In [179]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=2000)

## 1. Logistic Regression

In [180]:
lr = LogisticRegression(featuresCol="features", labelCol="label", elasticNetParam=0.5)
lr_model = lr.fit(train_data)

train_predictions = lr_model.transform(train_data)
test_predictions = lr_model.transform(test_data)

In [181]:
def eval(model_name, train_predictions, test_predictions):
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
    bin_avaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction")

    # Accuracy
    train_acc = evaluator.evaluate(train_predictions, {evaluator.metricName: "accuracy"})
    test_acc = evaluator.evaluate(test_predictions, {evaluator.metricName: "accuracy"})

    # F1-score
    train_f1 = evaluator.evaluate(train_predictions, {evaluator.metricName: "f1"})
    test_f1 = evaluator.evaluate(test_predictions, {evaluator.metricName: "f1"})

    # AUC
    train_auc = bin_avaluator.evaluate(train_predictions)
    test_auc = bin_avaluator.evaluate(test_predictions)

    return {
        "model_name": model_name,
        "train_accuracy": train_acc,
        "test_accuracy": test_acc,
        "train_f1": train_f1,
        "test_f1": test_f1,
        "train_auc": train_auc,
        "test_auc": test_auc
        }
# print(f"Train Accuracy: {train_acc}, Train F1-Score: {train_f1}, Train AUC: {train_auc}")
# print(f"Test Accuracy: {test_acc}, Test F1-Score: {test_f1}, Test AUC: {test_auc}")

In [182]:
lr_metrics = eval("Logistic regresstion", train_predictions, test_predictions)
print(lr_metrics)

{'model_name': 'Logistic regresstion', 'train_accuracy': 0.8251133031191682, 'test_accuracy': 0.8110072689511942, 'train_f1': 0.8222438439400226, 'test_f1': 0.808016474890826, 'train_auc': 0.8553104677121898, 'test_auc': 0.8430894646227887}


## 2. Decision Tree

In [183]:
dc = DecisionTreeClassifier(featuresCol="features", labelCol="label", maxDepth=5)
dc_model = dc.fit(train_data)

train_predictions = dc_model.transform(train_data)
test_predictions = dc_model.transform(test_data)

In [184]:
dc_metrics = eval("Decision Tree", train_predictions, test_predictions)
print(dc_metrics)

{'model_name': 'Decision Tree', 'train_accuracy': 0.8800319914689416, 'test_accuracy': 0.8681204569055037, 'train_f1': 0.8768901108847249, 'test_f1': 0.8656174950523802, 'train_auc': 0.6707551617510434, 'test_auc': 0.6186344873363895}


## Random Forest

In [185]:
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=20)
rf_model = rf.fit(train_data)

train_predictions = rf_model.transform(train_data)
test_predictions = rf_model.transform(test_data)

In [186]:
rf_metrics = eval("Random Forest", train_predictions, test_predictions)
print(rf_metrics)

{'model_name': 'Random Forest', 'train_accuracy': 0.8829645427885897, 'test_accuracy': 0.8712357217030114, 'train_f1': 0.8789910473219407, 'test_f1': 0.8673075270621999, 'train_auc': 0.9258361412704355, 'test_auc': 0.929766744798689}


## Gradient Boosting Tree

In [187]:
gb = GBTClassifier(featuresCol="features", labelCol="label", maxIter=10)
gb_model = gb.fit(train_data)

train_predictions = gb_model.transform(train_data)
test_predictions = gb_model.transform(test_data)

In [188]:
gb_metrics = eval("Gradient Boosting", train_predictions, test_predictions)
print(gb_metrics)

{'model_name': 'Gradient Boosting', 'train_accuracy': 0.8938949613436417, 'test_accuracy': 0.8764278296988577, 'train_f1': 0.8912704342614751, 'test_f1': 0.8742399903157332, 'train_auc': 0.9483282374474814, 'test_auc': 0.9409550084009211}


In [189]:
import pandas as pd

metrics = [lr_metrics, dc_metrics, rf_metrics, gb_metrics]
df_metrics = pd.DataFrame(metrics)
df_metrics.head()

,model_name,train_accuracy,test_accuracy,train_f1,test_f1,train_auc,test_auc
0,Logistic regresstion,0.825113,0.811007,0.822244,0.808016,0.855310,0.843089
1,Decision Tree,0.880032,0.868120,0.876890,0.865617,0.670755,0.618634
2,Random Forest,0.882965,0.871236,0.878991,0.867308,0.925836,0.929767
3,Gradient Boosting,0.893895,0.876428,0.891270,0.874240,0.948328,0.940955


Regarding the globals performances based on the differents metrics such as **Accuracy**, **f1-score** and **AUC**, we will choose `Gradient Boosting` that shows greats performances.

In [190]:
# save the best model to the current directory: gradient boosting model
gb_model.write().overwrite().save("./bestmodel")

# Part 2

In [191]:
from pyspark.sql.window import Window

In [192]:
sample_data = spark.read.csv("./dataset_sample_spark.csv", sep=";", header=True)

In [193]:
# suppression des doublons en gardant
sample_data = sample_data.dropDuplicates(["sensor", "servicePoint", "transmitter", "date"])
sample_data.show(5)

+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+
|     sensor|servicePoint|   client|transmitter|      date|meterDiameter|volume_l_value|index_interpolated_d_l_value|
+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|01/01/2019|           40|           0.0|                   5623306.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|01/01/2020|           40|          95.0|                   5991416.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|01/01/2021|           40|           7.0|                   6713307.5|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|01/02/2019|           40|          15.0|                   5648016.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|01/02/2020|           40|         339.0|                   6026507.0|
+-----------+------------+---------+-----------+--------

## Calcul de la consommation moyenne par diamètre de compteur

In [194]:
# consommation moyenne par diamètre
avg_consumption_by_diameter = sample_data.groupBy(["sensor", "meterDiameter"]).agg(f.mean("volume_l_value").alias("avg_consumption"))
avg_consumption_by_diameter.show()

+-----------+-------------+------------------+
|     sensor|meterDiameter|   avg_consumption|
+-----------+-------------+------------------+
|C10FA046664|           15|116.76915487907694|
|C08EB001382|           20| 99.49573222647919|
|C09FA079673|           15| 1018.235997442455|
|C10ED005356|           30| 629.8111048051459|
|C08FA041975|           15| 60.94796407185634|
|C09FA127701|           15|153.78950403690888|
|C08FA085851|           15| 210.2792194642136|
|C07AA092853|           15| 766.9642578125004|
|C09FA030008|           15|  563.314566115702|
|C04AE134021|           40|1529.3411509188334|
+-----------+-------------+------------------+



In [195]:
# gros compteur
global_avg_consumption = sample_data.agg(f.mean("volume_l_value")).collect()[0][0]
gros_compteurs = avg_consumption_by_diameter.filter(f.col("avg_consumption") > global_avg_consumption)
gros_compteurs.show()

+-----------+-------------+------------------+
|     sensor|meterDiameter|   avg_consumption|
+-----------+-------------+------------------+
|C09FA079673|           15| 1018.235997442455|
|C10ED005356|           30| 629.8111048051459|
|C07AA092853|           15| 766.9642578125004|
|C09FA030008|           15|  563.314566115702|
|C04AE134021|           40|1529.3411509188334|
+-----------+-------------+------------------+



## compute V1

In [196]:
# 1. casting date format
df = sample_data.withColumn("date", f.to_date(f.col("date"), "dd/MM/yyyy"))

# Calcul de T : date maximale pour chaque triplette unique [sensor, servicePoint, client]
df_t = df.groupBy("sensor", "servicePoint", "client").agg(
    f.max("date").alias("T")
)

# Joindre T avec le DataFrame initial
df = df.join(df_t, on=["sensor", "servicePoint", "client"], how="left")

# 2. compute significant concumption column: where volume > 20.0 litters
df = df.withColumn(
    "significant_consumption",
    f.when(f.col("volume_l_value").cast("double") > 20, 1).otherwise(0)
)

# 3 compute T0: max significant consumption date before current date 
window_spec_lag = Window.partitionBy("sensor", "servicePoint").orderBy("date").rowsBetween(Window.unboundedPreceding, -1)

df = df.withColumn(
    "T0",
    f.max(
        f.when(
            f.col("significant_consumption") == 1,
            f.col("date")
        )
    ).over(window_spec_lag)
)

# compute v1
df = df.withColumn(
    "V1",
    f.when(
        f.col("T0").isNotNull(),
        f.datediff(
            f.col("T"),
            f.col("T0")
        )
    ).otherwise(730)
)

# # Afficher le résultat
df.select("sensor", "date", "volume_l_value", "V1").show(10)

+-----------+----------+--------------+---+
|     sensor|      date|volume_l_value| V1|
+-----------+----------+--------------+---+
|C04AE134021|2019-01-01|           0.0|730|
|C04AE134021|2019-01-02|          61.0|730|
|C04AE134021|2019-01-03|           7.0|949|
|C04AE134021|2019-01-04|           5.0|949|
|C04AE134021|2019-01-05|           3.0|949|
|C04AE134021|2019-01-06|        1200.0|949|
|C04AE134021|2019-01-07|        1076.0|945|
|C04AE134021|2019-01-08|        2346.0|944|
|C04AE134021|2019-01-09|        4533.0|943|
|C04AE134021|2019-01-10|        1165.0|942|
+-----------+----------+--------------+---+
only showing top 10 rows



In [197]:
df.select("V1").distinct().show(10)

+---+
| V1|
+---+
|833|
|496|
|463|
|148|
|471|
|858|
|737|
|623|
|540|
|392|
+---+
only showing top 10 rows



## compute V2

In [198]:
window_spec = Window.partitionBy("sensor", "servicePoint", "transmitter").orderBy(f.desc("date"))

# Trouver la dernière date avec une consommation significative
df = df.withColumn(
    "last_significant_date",
    f.when(f.col("significant_consumption") == 1, f.col("date"))
).withColumn(
    "last_significant_date",
    f.last("last_significant_date", ignorenulls=True).over(window_spec)
)
df = df.withColumn(
    "T0", f.last("last_significant_date", ignorenulls=True).over(window_spec)
)

# Filtrer les 90 jours avant T0
df = df.withColumn(
    "within_90_days",
    f.when(
        (f.col("date") <= f.col("T0")) & (f.col("date") > f.date_sub(f.col("T0"), 90)),
        1
    ).otherwise(0)
)

# Calculer le nombre de jours avec consommation non nulle dans les 90 jours avant T0
df = df.withColumn(
    "non_null_days",
    f.when((f.col("within_90_days") == 1) & (f.col("volume_l_value") > 0), 1).otherwise(0)
)
df = df.withColumn(
    "total_non_null_days", f.sum("non_null_days").over(window_spec)
)
# Calculer v2
df = df.withColumn(
    "v2", f.col("total_non_null_days") / 90
)
df.select("sensor", "date", "volume_l_value", "V2").show(5)

+-----------+----------+--------------+---+
|     sensor|      date|volume_l_value| V2|
+-----------+----------+--------------+---+
|C04AE134021|2021-08-08|           0.0|0.0|
|C04AE134021|2021-08-07|           0.0|0.0|
|C04AE134021|2021-08-06|           0.0|0.0|
|C04AE134021|2021-08-05|           0.0|0.0|
|C04AE134021|2021-08-04|           0.0|0.0|
+-----------+----------+--------------+---+
only showing top 5 rows



In [199]:
df.select("V2").distinct().show(10)

+-------------------+
|                 V2|
+-------------------+
| 7.2444444444444445|
|  7.533333333333333|
|  7.711111111111111|
|  7.866666666666666|
|  9.688888888888888|
|0.18888888888888888|
| 1.2666666666666666|
| 1.7444444444444445|
| 2.5444444444444443|
|  4.877777777777778|
+-------------------+
only showing top 10 rows



## Compute V3

In [200]:
# Définir la période de 6 mois (183 jours) avant T0 - 90 jours
df = df.withColumn(
    "within_183_days",
    f.when(
        (f.col("date") <= f.date_sub(f.col("T0"), 90)) & 
        (f.col("date") > f.date_sub(f.col("T0"), 273)), 
        1
    ).otherwise(0)
)

# Calculer le nombre de jours avec consommation non nulle dans la période de 6 mois
df = df.withColumn(
    "non_null_days_v3",
    f.when((f.col("within_183_days") == 1) & (f.col("volume_l_value") > 0), 1).otherwise(0)
)

df = df.withColumn(
    "total_non_null_days_v3", 
    f.sum("non_null_days_v3").over(Window.partitionBy("sensor", "servicePoint", "transmitter"))
)

# Calculer v3
df = df.withColumn(
    "v3", f.col("total_non_null_days_v3") / 183
)
df.select("sensor", "date", "volume_l_value", "V3").show(5)


+-----------+----------+--------------+---+
|     sensor|      date|volume_l_value| V3|
+-----------+----------+--------------+---+
|C04AE134021|2021-08-08|           0.0|0.0|
|C04AE134021|2021-08-07|           0.0|0.0|
|C04AE134021|2021-08-06|           0.0|0.0|
|C04AE134021|2021-08-05|           0.0|0.0|
|C04AE134021|2021-08-04|           0.0|0.0|
+-----------+----------+--------------+---+
only showing top 5 rows



## Compute V4

In [201]:
# Filtrer les données sur une période d'un an avant T0
df = df.withColumn(
    "within_1_year",
    f.when((f.col("date") <= f.col("T0")) & (f.col("date") > f.date_sub(f.col("T0"), 365)), 1).otherwise(0)
)

# Marquer les jours avec consommation nulle
df = df.withColumn(
    "is_null", f.when((f.col("within_1_year") == 1) & (f.col("volume_l_value") == 0), 1).otherwise(0)
)

# Identifier les débuts et fins de périodes de consommation nulle
window_spec = Window.partitionBy("sensor", "servicePoint", "transmitter").orderBy("date")
df = df.withColumn(
    "prev_is_null", f.lag("is_null").over(window_spec)
)
df = df.withColumn(
    "new_null_period",
    f.when((f.col("is_null") == 1) & ((f.col("prev_is_null").isNull()) | (f.col("prev_is_null") == 0)), 1).otherwise(0)
)

# Attribuer un identifiant unique à chaque période nulle
df = df.withColumn(
    "null_period_id",
    f.sum("new_null_period").over(window_spec)
)

# Calculer la longueur de chaque période nulle
null_period_lengths = df.filter(f.col("is_null") == 1).groupBy(
    "sensor", "servicePoint", "transmitter", "null_period_id"
).agg(f.count("date").alias("period_length"))

# Calculer la durée moyenne des périodes nulles
avg_null_period_length = null_period_lengths.groupBy(
    "sensor", "servicePoint", "transmitter"
).agg(f.avg("period_length").alias("V4"))

# Joindre la colonne v4 au dataset principal
df = df.join(avg_null_period_length, ["sensor", "servicePoint", "transmitter"], "left")

df.select("sensor", "date", "volume_l_value", "V4").show(5)

+-----------+----------+--------------+------------------+
|     sensor|      date|volume_l_value|                V4|
+-----------+----------+--------------+------------------+
|C04AE134021|2019-01-01|           0.0|1.8918918918918919|
|C04AE134021|2020-01-01|          95.0|1.8918918918918919|
|C04AE134021|2021-01-01|           7.0|1.8918918918918919|
|C04AE134021|2019-02-01|          15.0|1.8918918918918919|
|C04AE134021|2020-02-01|         339.0|1.8918918918918919|
+-----------+----------+--------------+------------------+
only showing top 5 rows



In [202]:
df.select("V4").distinct().show(10)

+------------------+
|                V4|
+------------------+
|               8.0|
| 9.090909090909092|
|              16.7|
|2.3333333333333335|
|1.8918918918918919|
| 5.434782608695652|
|3.6206896551724137|
|              7.75|
|               3.3|
|              NULL|
+------------------+



## Compute V5

In [203]:
# Identifier les jours de consommation nulle
df = df.withColumn(
    "is_null", f.when((f.col("within_1_year") == 1) & (f.col("volume_l_value") == 0), 1).otherwise(0)
)

# Identifier les débuts de périodes nulles
df = df.withColumn(
    "prev_is_null", f.lag("is_null").over(Window.partitionBy("sensor", "servicePoint", "transmitter").orderBy("date"))
)
df = df.withColumn(
    "new_null_period",
    f.when((f.col("is_null") == 1) & ((f.col("prev_is_null").isNull()) | (f.col("prev_is_null") == 0)), 1).otherwise(0)
)

# Identifier chaque période de consommation nulle avec un ID unique
df = df.withColumn(
    "null_period_id",
    f.sum("new_null_period").over(Window.partitionBy("sensor", "servicePoint", "transmitter").orderBy("date"))
)

# Calculer la longueur de chaque période nulle
null_period_lengths = df.filter(f.col("is_null") == 1).groupBy(
    "sensor", "servicePoint", "transmitter", "null_period_id"
).agg(f.count("date").alias("period_length"))

# Trouver la durée maximale des périodes nulles (v5)
max_null_period_length = null_period_lengths.groupBy(
    "sensor", "servicePoint", "transmitter"
).agg(f.max("period_length").alias("V5"))

# Joindre la colonne v5 au dataset principal
df = df.join(max_null_period_length, ["sensor", "servicePoint", "transmitter"], "left")

df.select("sensor", "date", "volume_l_value", "V5").show(5)

+-----------+----------+--------------+---+
|     sensor|      date|volume_l_value| V5|
+-----------+----------+--------------+---+
|C04AE134021|2019-01-01|           0.0|  6|
|C04AE134021|2020-01-01|          95.0|  6|
|C04AE134021|2021-01-01|           7.0|  6|
|C04AE134021|2019-02-01|          15.0|  6|
|C04AE134021|2020-02-01|         339.0|  6|
+-----------+----------+--------------+---+
only showing top 5 rows



In [204]:
df.select("V5").distinct().show(10)

+----+
|  V5|
+----+
| 113|
|  50|
|  25|
|   6|
|   9|
|  89|
|  48|
|  12|
|  21|
|NULL|
+----+



## Compute V6

In [205]:
# Identifier les jours de consommation non nulle
df = df.withColumn(
    "is_non_null", f.when((f.col("within_1_year") == 1) & (f.col("volume_l_value") > 0), 1).otherwise(0)
)

# Identifier les débuts de périodes non nulles
df = df.withColumn(
    "prev_is_non_null", f.lag("is_non_null").over(Window.partitionBy("sensor", "servicePoint", "transmitter").orderBy("date"))
)
df = df.withColumn(
    "new_non_null_period",
    f.when((f.col("is_non_null") == 1) & ((f.col("prev_is_non_null").isNull()) | (f.col("prev_is_non_null") == 0)), 1).otherwise(0)
)

# Identifier chaque période de consommation non nulle avec un ID unique
df = df.withColumn(
    "non_null_period_id",
    f.sum("new_non_null_period").over(Window.partitionBy("sensor", "servicePoint", "transmitter").orderBy("date"))
)

# Calculer la longueur de chaque période non nulle
non_null_period_lengths = df.filter(f.col("is_non_null") == 1).groupBy(
    "sensor", "servicePoint", "transmitter", "non_null_period_id"
).agg(f.count("date").alias("non_null_period_length"))

# Trouver la durée maximale des périodes non nulles (v6)
max_non_null_period_length = non_null_period_lengths.groupBy(
    "sensor", "servicePoint", "transmitter"
).agg(f.max("non_null_period_length").alias("V6"))

# Joindre la colonne v6 au dataset principal
df = df.join(max_non_null_period_length, ["sensor", "servicePoint", "transmitter"], "left")

df.select("sensor", "date", "volume_l_value", "V6").show(5)

+-----------+----------+--------------+---+
|     sensor|      date|volume_l_value| V6|
+-----------+----------+--------------+---+
|C04AE134021|2019-01-01|           0.0|250|
|C04AE134021|2020-01-01|          95.0|250|
|C04AE134021|2021-01-01|           7.0|250|
|C04AE134021|2019-02-01|          15.0|250|
|C04AE134021|2020-02-01|         339.0|250|
+-----------+----------+--------------+---+
only showing top 5 rows



In [206]:
df.select("V6").distinct().show(10)

+---+
| V6|
+---+
|270|
|858|
|250|
|375|
|301|
|540|
|323|
|176|
|117|
| 91|
+---+



## Compute V7

In [207]:
# Identifier l'index mécanique à la dernière consommation significative (T0)
window_spec = Window.partitionBy("sensor", "servicePoint", "transmitter").orderBy("date").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

# Filtrer pour récupérer la valeur de l'index à T0
df = df.withColumn(
    "v7",
    f.when(f.col("date") == f.col("T0"), f.col("index_interpolated_d_l_value")).otherwise(None)
)

# Propager l'index mécanique (v7) pour chaque triplette
df = df.withColumn(
    "v7", f.last("v7", ignorenulls=True).over(window_spec)
)

df.select("sensor", "date", "volume_l_value", "V7").show(5)

+-----------+----------+--------------+---------+
|     sensor|      date|volume_l_value|       V7|
+-----------+----------+--------------+---------+
|C04AE134021|2019-01-01|           0.0|7070442.0|
|C04AE134021|2019-01-02|          61.0|7070442.0|
|C04AE134021|2019-01-03|           7.0|7070442.0|
|C04AE134021|2019-01-04|           5.0|7070442.0|
|C04AE134021|2019-01-05|           3.0|7070442.0|
+-----------+----------+--------------+---------+
only showing top 5 rows



In [208]:
df.select("V7").distinct().show(10)

+---------+
|       V7|
+---------+
| 426294.0|
|2811627.0|
|3526318.0|
|7070442.0|
| 543614.0|
|2252939.5|
|1694238.0|
| 600791.2|
|1208589.0|
|2100873.0|
+---------+



## Compute V8

In [209]:

# Identifier les jours de consommation non nulle dans l'année précédant T0
df = df.withColumn(
    "is_non_null", f.when((f.col("within_1_year") == 1) & (f.col("volume_l_value") > 0), 1).otherwise(0)
)

# Identifier les débuts de périodes non nulles
window_spec = Window.partitionBy("sensor", "servicePoint", "transmitter").orderBy("date")
df = df.withColumn(
    "prev_is_non_null", f.lag("is_non_null").over(window_spec)
)
df = df.withColumn(
    "new_non_null_period",
    f.when((f.col("is_non_null") == 1) & ((f.col("prev_is_non_null").isNull()) | (f.col("prev_is_non_null") == 0)), 1).otherwise(0)
)

# Calculer le nombre total de périodes non nulles (v8)
df = df.withColumn(
    "V8", f.sum("new_non_null_period").over(Window.partitionBy("sensor", "servicePoint", "transmitter"))
)

df.select("sensor", "date", "volume_l_value", "V8").show(5)

+-----------+----------+--------------+---+
|     sensor|      date|volume_l_value| V8|
+-----------+----------+--------------+---+
|C04AE134021|2019-01-01|           0.0| 37|
|C04AE134021|2019-01-02|          61.0| 37|
|C04AE134021|2019-01-03|           7.0| 37|
|C04AE134021|2019-01-04|           5.0| 37|
|C04AE134021|2019-01-05|           3.0| 37|
+-----------+----------+--------------+---+
only showing top 5 rows



In [210]:
df.select("V8").distinct().show(10)

+---+
| V8|
+---+
| 34|
| 33|
|  5|
|  1|
| 37|
| 11|
| 30|
| 91|
| 47|
+---+



## Compute V9

In [211]:
consumption_features = ["V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9"]
millesime_0 = ['C10FA','C10LA','C10SA', 'C11FA','C11LA','C11SA']
millesime_1 = ['D16BU', 'Z12ER', 'C07AA']

# Extraire les 5 premiers caractères du champ sensor pour obtenir le millesime
df = df.withColumn("Millesime", f.substring(f.col("sensor"), 1, 5))

df = df.withColumn(
    'V9',
    f.when(
        (df['Millesime'].isin(millesime_0)) & (f.length(f.col('V7').cast("string"))>3),
        0
    ).otherwise(
        f.when(
            df['Millesime'].isin(millesime_1),
            1
        ).otherwise(2)
    )
)
df.select("sensor", "date", "volume_l_value", "Millesime", "V7", "V9").show(10)

+-----------+----------+--------------+---------+---------+---+
|     sensor|      date|volume_l_value|Millesime|       V7| V9|
+-----------+----------+--------------+---------+---------+---+
|C04AE134021|2019-01-01|           0.0|    C04AE|7070442.0|  2|
|C04AE134021|2019-01-02|          61.0|    C04AE|7070442.0|  2|
|C04AE134021|2019-01-03|           7.0|    C04AE|7070442.0|  2|
|C04AE134021|2019-01-04|           5.0|    C04AE|7070442.0|  2|
|C04AE134021|2019-01-05|           3.0|    C04AE|7070442.0|  2|
|C04AE134021|2019-01-06|        1200.0|    C04AE|7070442.0|  2|
|C04AE134021|2019-01-07|        1076.0|    C04AE|7070442.0|  2|
|C04AE134021|2019-01-08|        2346.0|    C04AE|7070442.0|  2|
|C04AE134021|2019-01-09|        4533.0|    C04AE|7070442.0|  2|
|C04AE134021|2019-01-10|        1165.0|    C04AE|7070442.0|  2|
+-----------+----------+--------------+---------+---------+---+
only showing top 10 rows



In [212]:
df.select("V9").distinct().show(10)

+---+
| V9|
+---+
|  1|
|  2|
|  0|
+---+



##  Make predictons

In [213]:
consumption_features = ["V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9"]

# df = df.select(consumption_features)
df = cast_col(df, consumption_features, "integer")
df = df.dropna()
df.show()

+-----------+------------+-----------+---------+----------+-------------+--------------+----------------------------+----------+-----------------------+----------+---+---------------------+--------------+-------------+-------------------+---+---------------+----------------+----------------------+---+-------------+-------+------------+---------------+--------------+---+---+-----------+----------------+-------------------+------------------+---+-------+---+---------+---+
|     sensor|servicePoint|transmitter|   client|      date|meterDiameter|volume_l_value|index_interpolated_d_l_value|         T|significant_consumption|        T0| V1|last_significant_date|within_90_days|non_null_days|total_non_null_days| V2|within_183_days|non_null_days_v3|total_non_null_days_v3| V3|within_1_year|is_null|prev_is_null|new_null_period|null_period_id| V4| V5|is_non_null|prev_is_non_null|new_non_null_period|non_null_period_id| V6|     V7| V8|Millesime| V9|
+-----------+------------+-----------+---------+--

## Load best model for predicition

In [214]:
from pyspark.ml.classification import GBTClassificationModel

# Load model
loaded_model = GBTClassificationModel.load("./bestmodel")

In [215]:
# Créer le VectorAssembler
assembler = VectorAssembler(inputCols=consumption_features, outputCol="features")

# Appliquer la transformation pour créer la colonne "features"
X_features = assembler.transform(df)

# Afficher les données transformées
X_features.show()

+-----------+------------+-----------+---------+----------+-------------+--------------+----------------------------+----------+-----------------------+----------+---+---------------------+--------------+-------------+-------------------+---+---------------+----------------+----------------------+---+-------------+-------+------------+---------------+--------------+---+---+-----------+----------------+-------------------+------------------+---+-------+---+---------+---+--------------------+
|     sensor|servicePoint|transmitter|   client|      date|meterDiameter|volume_l_value|index_interpolated_d_l_value|         T|significant_consumption|        T0| V1|last_significant_date|within_90_days|non_null_days|total_non_null_days| V2|within_183_days|non_null_days_v3|total_non_null_days_v3| V3|within_1_year|is_null|prev_is_null|new_null_period|null_period_id| V4| V5|is_non_null|prev_is_non_null|new_non_null_period|non_null_period_id| V6|     V7| V8|Millesime| V9|            features|
+-------

In [216]:
# make predicition on new data
predictions = gb_model.transform(X_features)

predictions.select("features", "prediction", "probability").show()

+--------------------+----------+--------------------+
|            features|prediction|         probability|
+--------------------+----------+--------------------+
|[730.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[949.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[949.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[949.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[949.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[945.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[944.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[943.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[942.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[941.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[941.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[941.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[938.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[937.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[936.0,9.0,0.0,1....|       0.0|[0.97984154434007...|
|[935.0,9.

## Save predicitons into CSV file

In [217]:
# - BLOCKED: 0  
# - STOPPED: 1
predictions = predictions.withColumn(
    "predicited_label",
    f.when(
        f.col("prediction") == 0.0,
        "BLOCKED"
    ).otherwise("STOPPED")
)

selected_columns = ["sensor", "servicePoint", "transmitter", "date", "volume_l_value", "V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "predicited_label"]
predictions = predictions.select(selected_columns)


# save predicitions into csv file
predictions.coalesce(1).write.csv("./predictions", header=True, mode="overwrite")

In [218]:
predictions.groupBy("predicited_label").agg(f.count("*")).show()

+----------------+--------+
|predicited_label|count(1)|
+----------------+--------+
|         BLOCKED|    4983|
|         STOPPED|    2171|
+----------------+--------+

